# Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import module.influx_utils as mod_inf
import pandas as pd
import fs
import os

# Change Time

In [3]:
# Example usage
ticks = 133539005217964000
print(mod_inf.ticks_to_rfc3339(ticks))  # Output: 2020-12-18T18:16:11Z

2024-03-03T00:48:41.796401Z


# Directories

In [4]:
MAIN_DIR = fs.open_fs("../data/processed/final")
DIR_TLG_SF_CSV = MAIN_DIR.getsyspath("TLG_SF.csv")
DIR_TLG_SF_CSV_INF = MAIN_DIR.getsyspath("TLG_SF_INF.csv")
DIR_TLG_SF_CSV_INF_temp = MAIN_DIR.getsyspath("TLG_SF_INF_temp.csv")

In [5]:
dF_TLG_SF = pd.read_csv(DIR_TLG_SF_CSV)
dF_TLG_SF.head()

,pk_TimeStamp,Value,tag_name
0,133543067468902405,12210.074219,Totalizador_Transferencia
1,133543079289260621,8033.640137,Totalizador_Transferencia
2,133543111229257914,4964.895996,Totalizador_Transferencia
3,133543134489783109,5118.520508,Totalizador_Transferencia
4,133543161780470776,5799.258301,Totalizador_Transferencia


In [6]:
dF_TLG_SF['_time'] = dF_TLG_SF['pk_TimeStamp'].apply(lambda x: mod_inf.ticks_to_rfc3339(x))

In [7]:
dF_TLG_SF.head()

,pk_TimeStamp,Value,tag_name,_time
0,133543067468902405,12210.074219,Totalizador_Transferencia,2024-03-07T17:39:06.890241Z
1,133543079289260621,8033.640137,Totalizador_Transferencia,2024-03-07T17:58:48.926062Z
2,133543111229257914,4964.895996,Totalizador_Transferencia,2024-03-07T18:52:02.925791Z
3,133543134489783109,5118.520508,Totalizador_Transferencia,2024-03-07T19:30:48.978313Z
4,133543161780470776,5799.258301,Totalizador_Transferencia,2024-03-07T20:16:18.047077Z


In [8]:
dF_TLG_SF_INF = dF_TLG_SF[['_time', 'Value', 'tag_name']].copy()
dF_TLG_SF_INF.columns = ['_time', '_value', '_field']
dF_TLG_SF_INF.to_csv(DIR_TLG_SF_CSV_INF, encoding='utf-8', index=False)
dF_TLG_SF_INF

,_time,_value,_field
0,2024-03-07T17:39:06.890241Z,12210.074219,Totalizador_Transferencia
1,2024-03-07T17:58:48.926062Z,8033.640137,Totalizador_Transferencia
2,2024-03-07T18:52:02.925791Z,4964.895996,Totalizador_Transferencia
3,2024-03-07T19:30:48.978313Z,5118.520508,Totalizador_Transferencia
4,2024-03-07T20:16:18.047077Z,5799.258301,Totalizador_Transferencia
...,...,...,...
535263,2020-10-23T16:00:48.143311Z,25.374699,TempSiloSQ
535264,2020-10-23T16:00:51.147592Z,25.371086,TempSiloSQ
535265,2020-10-23T16:00:52.146400Z,25.374699,TempSiloSQ
535266,2020-10-23T16:00:54.150870Z,25.381926,TempSiloSQ


In [9]:
dF_TLG_SF_INF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535268 entries, 0 to 535267
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   _time   535268 non-null  object 
 1   _value  535268 non-null  float64
 2   _field  535268 non-null  object 
dtypes: float64(1), object(2)
memory usage: 12.3+ MB


In [10]:
measurement : str  = "sensorData5"
headers: str = f"#constant measurement,{measurement}\n{mod_inf.generate_headers_from_dataframe(dF_TLG_SF_INF)}"
file_path = DIR_TLG_SF_CSV_INF
temp_file_path = DIR_TLG_SF_CSV_INF_temp

# Read the existing content of the file
with open(file_path, 'r') as original_file:
    existing_content = original_file.read()

# Write the headers and then the existing content to a temporary file
with open(temp_file_path, 'w') as temp_file:
    temp_file.write(headers + '\n')
    temp_file.write(existing_content)

# Replace the original file with the temporary file
os.replace(temp_file_path, file_path)